In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
import pandas as pd
print(pd.read_csv("/content/drive/MyDrive/data4you/W3_D1/dash_data/AAPL_funda.csv"))

   Unnamed: 0 Instrument  Earnings before Interest & Taxes (EBIT)  \
0           0     AAPL.O                             113965000000   
1           1     AAPL.O                             112304000000   
2           2     AAPL.O                             112226000000   
3           3     AAPL.O                             114301000000   
4           4     AAPL.O                             118658000000   
5           5     AAPL.O                             118240000000   

  Period End Date   Calc Date  
0      2022-12-31  2023-02-02  
1      2023-04-01  2023-05-04  
2      2023-07-01  2023-08-03  
3      2023-09-30  2023-11-02  
4      2023-12-30  2024-02-01  
5      2024-03-30  2024-05-02  


In [34]:

selected_stock_value = "AAPL"
funda_orig_df = read_csv(filepath_or_buffer=f"/content/drive/MyDrive/data4you/W3_D1/dash_data/{selected_stock_value}_funda.csv")
funda_orig_df.drop(columns=['Unnamed: 0'], inplace=True)

datecols = ["Calc Date", "Period End Date"]
# Convert date columns
for datecol in datecols:
    funda_orig_df[datecol] = to_datetime(funda_orig_df[datecol])

funda_orig_df['year'] = funda_orig_df["Period End Date"].dt.year.astype(str).str[2:]
funda_orig_df['quarter'] = funda_orig_df["Period End Date"].dt.quarter.astype(str)
funda_orig_df['quarter_str'] = 'Q' + funda_orig_df['quarter'] + "'" + funda_orig_df['year']
funda_orig_df.rename(columns={'quarter_str': 'quarter'}, inplace=True)

# Process funda2_df
funda2_orig_df = read_csv(filepath_or_buffer=f"/content/drive/MyDrive/data4you/W3_D1/dash_data/{selected_stock_value}_funda2.csv")
funda2_orig_df.drop(columns=["Instrument"], inplace=True)
funda2_orig_df["Date"] = to_datetime(funda2_orig_df["Date"])
funda2_orig_df.drop_duplicates(subset=["Date"], keep='first', inplace=True)
funda2_df = funda2_orig_df.copy()
funda2_df.set_index(keys=["Date"], inplace=True)
funda2_df = funda2_df.resample('min').ffill().copy()
funda2_df.index = funda2_df.index.tz_convert(tz=None)

funda2_orig_df['year'] = funda2_orig_df["Date"].dt.year.astype(str).str[2:]
funda2_orig_df['quarter'] = funda2_orig_df["Date"].dt.quarter.astype(str)
funda2_orig_df['quarter_str'] = 'Q' + funda2_orig_df['quarter'] + "'" + funda2_orig_df['year']
# funda2_orig_df.rename(columns={'quarter_str': 'quarter'}, inplace=True)

print(funda2_orig_df['quarter'])
print(funda2_orig_df['quarter'].unique())

top_right_plot = make_subplots(specs=[[{"secondary_y": True}]])
top_right_plot.add_trace(Bar(
    name="EBIT",
    x=funda_orig_df["quarter"],
    y=funda_orig_df["Earnings before Interest & Taxes (EBIT)"]
), secondary_y=False)
for quarter in funda2_orig_df['quarter'].unique():
    top_right_plot.add_trace(Box(
        y=funda2_orig_df[funda2_orig_df['quarter'] == quarter]['P/E (Daily Time Series Ratio)'],
        name=quarter,
        showlegend=False
    ), secondary_y=True)


0      1
1      1
2      1
3      1
4      1
      ..
372    2
373    2
374    3
375    3
376    3
Name: quarter, Length: 377, dtype: object
['1' '2' '3' '4']


In [35]:
!pip install dash
!pip install plotly
# Import necessary libraries
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from pandas import DataFrame, Timestamp, concat, read_csv, to_datetime, NA
from plotly.graph_objects import Bar, Box, Candlestick, Figure, Scatter
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler

# Define utility colors
colors = {
    'background_maindiv': '#F3FF33',
    'background_plot': '#B4BD26'
}

# Create a Dash application
app = Dash(__name__)

# Define the layout of the application
app.layout = html.Div(
    style={'backgroundColor': colors['background_maindiv']},
    children=[
        html.H1(
            children='Stock Dashboard',
            style={'textAlign': 'center', 'font-family': 'Helvetica', "fontSize": "48px"}
        ),
        html.Div(
            children=[
                dcc.Dropdown(['AAPL', 'AMZN', 'GOOG'], value='AAPL', id='selected-stock')
            ],
            style={'width': '100%', 'display': 'inline-block', 'verticalAlign': 'middle'}
        ),
        html.Div(
            children=[
                dcc.Graph(figure={}, id='our-first-graph', style={'display': 'inline-block'}),
                dcc.Graph(figure={}, id='our-second-graph', style={'display': 'inline-block'})
            ],
            style={'display': 'inline-block', 'verticalAlign': 'middle'}
        ),
        html.Div(
            children=[
                dcc.Graph(figure={}, id='our-third-graph')
            ],
            style={'width': '100%', 'display': 'inline-block', 'verticalAlign': 'middle'}
        )
    ]
)

# Define the callback function
@app.callback(
    [
        Output('our-first-graph', 'figure'),
        Output('our-second-graph', 'figure'),
        Output('our-third-graph', 'figure')
    ],
    [Input('selected-stock', 'value')]
)
def update_output(selected_stock_value):
    try:
        # Load data
        datecols = ["Calc Date", "Period End Date"]
        funda_orig_df = read_csv(filepath_or_buffer=f"/content/drive/MyDrive/data4you/W3_D1/dash_data/{selected_stock_value}_funda.csv")
        funda_orig_df.drop(columns=['Unnamed: 0'], inplace=True)

        # Convert date columns
        for datecol in datecols:
            funda_orig_df[datecol] = to_datetime(funda_orig_df[datecol])

        # Process funda_df
        funda_df = funda_orig_df.copy()
        funda_df.drop(columns=["Instrument", "Period End Date"], inplace=True)
        funda_df.set_index(keys=["Calc Date"], inplace=True)
        tmpadd = DataFrame(
            data={'Earnings before Interest & Taxes (EBIT)': [NA]},
            index=[Timestamp.today()]
        )
        tmpadd.index.rename(name='Calc Date', inplace=True)
        funda_df = concat([funda_df, tmpadd], axis=0)
        funda_df = funda_df.resample('min').ffill().copy()
        funda_df.dropna(inplace=True)
        funda_df.index.rename(name="Date", inplace=True)

        # Process funda2_df
        funda2_orig_df = read_csv(filepath_or_buffer=f"/content/drive/MyDrive/data4you/W3_D1/dash_data/{selected_stock_value}_funda2.csv")
        funda2_orig_df.drop(columns=["Instrument"], inplace=True)
        funda2_orig_df["Date"] = to_datetime(funda2_orig_df["Date"])
        funda2_orig_df.drop_duplicates(subset=["Date"], keep='first', inplace=True)
        funda2_df = funda2_orig_df.copy()
        funda2_df.set_index(keys=["Date"], inplace=True)
        funda2_df = funda2_df.resample('min').ffill().copy()
        funda2_df.index = funda2_df.index.tz_convert(tz=None)

        # Process price data
        price_files = [
            f"/content/drive/MyDrive/data4you/W3_D1/dash_data/{selected_stock_value}_{period}.csv"
            for period in [
                "2023_07_01_2023_09_30",
                "2023_10_01_2023_12_31",
                "2024_01_01_2024_03_06",
                "2024_03_07_2024_05_28",
                "2024_05_29_2024_07_04"
            ]
        ]
        price_df_list = [read_csv(f, index_col='Date', parse_dates=["Date"]) for f in price_files]
        price_df = concat(price_df_list, axis=0)

        # Combine all data
        fdf = concat([price_df, funda_df, funda2_df], axis=1)
        fdf.dropna(inplace=True)
        fdf = fdf.tail(10000).copy()

        # First Chart
        top_left_plot = Figure(data=[Candlestick(
            x=fdf.index,
            open=fdf['OPEN'],
            high=fdf['HIGH'],
            low=fdf['LOW'],
            close=fdf['CLOSE'],
            name='price'
        )])
        top_left_plot.update_layout(xaxis_rangeslider_visible=False)

        # Third Chart
        scaler = StandardScaler()
        pdf = DataFrame(scaler.fit_transform(fdf), columns=fdf.columns, index=fdf.index)
        bottom_plot = Figure()
        for col in ['CLOSE', 'Earnings before Interest & Taxes (EBIT)', 'P/E (Daily Time Series Ratio)']:
            bottom_plot.add_trace(Scatter(x=pdf.index, y=pdf[col], name=col))

        # Process quarterly data for top-right chart
        funda_orig_df['year'] = funda_orig_df["Period End Date"].dt.year.astype(str).str[2:]
        funda_orig_df['quarter'] = funda_orig_df["Period End Date"].dt.quarter.astype(str)
        funda_orig_df['quarter_str'] = 'Q' + funda_orig_df['quarter'] + "'" + funda_orig_df['year']
        funda_orig_df.rename(columns={'quarter_str': 'quarter'}, inplace=True)

        funda2_orig_df['year'] = funda2_orig_df["Date"].dt.year.astype(str).str[2:]
        funda2_orig_df['quarter'] = funda2_orig_df["Date"].dt.quarter.astype(str)
        funda2_orig_df['quarter_str'] = 'Q' + funda2_orig_df['quarter'] + "'" + funda2_orig_df['year']
        # funda2_orig_df.rename(columns={'quarter_str': 'quarter'}, inplace=True)

        # Second Chart
        top_right_plot = Figure()
        top_right_plot = make_subplots(specs=[[{"secondary_y": True}]])
        top_right_plot.add_trace(Bar(
            name="EBIT",
            x=funda_orig_df["quarter"],
            y=funda_orig_df["Earnings before Interest & Taxes (EBIT)"]
        ), secondary_y=False)
        for quarter in funda2_orig_df.quarter.unique():
            top_right_plot.add_trace(Box(
                y=funda2_orig_df[funda2_orig_df['quarter'] == quarter]['P/E (Daily Time Series Ratio)'],
                name=quarter,
                showlegend=False
            ), secondary_y=True)

        return top_left_plot, top_right_plot, bottom_plot

    except FileNotFoundError as e:
        print(f"Error: {e}")
        return Figure(), Figure(), Figure()  # Return empty figures in case of missing files

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>